In [ ]:
import compyute as cp

In [ ]:
from compyute.preprocessing.text import CharacterTokenizer

with open("../data/tinyshakespeare.txt", "r") as f:
    data = f.read()

chars = sorted(list(set(data)))
tokenizer = CharacterTokenizer()
tokenizer.vocab = {i:c for i, c in enumerate(chars)}
tokenizer.ivocab = {c:i for i, c in enumerate(chars)}

In [ ]:
from transformer import get_causal_mask, Transformer

mask = get_causal_mask((256, 256))

model = Transformer(
    n_embeddings=tokenizer.vocab_size,
    embedding_dim=384,
    ffwd_channels=4*384,
    n_heads=6,
    n_blocks=6,
    max_seq_len=256,
    mask=mask
)
model.to_device(cp.cuda)

optim = cp.nn.optimizers.AdamW(model.get_parameters(), lr=3e-4)

state_dict = cp.load("shakespeare/transformer_shakespeare_5000.cp")
model.load_state_dict(state_dict["model"])
optim.load_state_dict(state_dict["optim"])

In [ ]:
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt

# get numpy array of embedding table
embs = model.token_emb.w.to_cpu()

# reduce dimensions to 2 to make
tsne = TSNE(random_state=0).fit_transform(embs)

# plot results
plt.figure(figsize=(8, 8))
plt.scatter(x=tsne[:,0], y=tsne[:,1], s=100)
for i in range(len(tsne)):
    char = tokenizer.decode(cp.tensor([i]))
    plt.text(x=tsne[i,0], y=tsne[i,1], s=char, ha="center", va="center", color="white")
plt.grid(linestyle="dashed", linewidth=0.5)

In [ ]:
import matplotlib.pyplot as plt
from transformer import  PositionalEncoding

pe = PositionalEncoding(256, 384, 1e2)

plt.imshow(pe.encodings.squeeze(), cmap="grey")